In [1]:
# 限制資料值的範圍
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from feature_engine.outliers import Winsorizer

C:\ProgramData\anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# load the California House price data from Scikit-learn
breast_cancer = load_breast_cancer()
X = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
y = breast_cancer.target

# let's separate the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

X_train.shape, X_test.shape

((398, 30), (171, 30))

In [3]:
X_train.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
478,11.490,14.59,73.99,404.9,0.10460,0.08228,0.05308,0.01969,0.1779,0.06574,...,12.40,21.90,82.04,467.6,0.1352,0.2010,0.25960,0.07431,0.2941,0.09180
303,10.490,18.61,66.86,334.3,0.10680,0.06678,0.02297,0.01780,0.1482,0.06600,...,11.06,24.54,70.76,375.4,0.1413,0.1044,0.08423,0.06528,0.2213,0.07842
155,12.250,17.94,78.27,460.3,0.08654,0.06679,0.03885,0.02331,0.1970,0.06228,...,13.59,25.22,86.60,564.2,0.1217,0.1788,0.19430,0.08211,0.3113,0.08132
186,18.310,18.58,118.60,1041.0,0.08588,0.08468,0.08169,0.05814,0.1621,0.05425,...,21.31,26.36,139.20,1410.0,0.1234,0.2445,0.35380,0.15710,0.3206,0.06938
101,6.981,13.43,43.79,143.5,0.11700,0.07568,0.00000,0.00000,0.1930,0.07818,...,7.93,19.54,50.41,185.2,0.1584,0.1202,0.00000,0.00000,0.2932,0.09382


In [4]:
def find_limits(df, variable, fold):
    lower_limit = df[variable].mean() - fold * df[variable].std()
    upper_limit = df[variable].mean() + fold * df[variable].std()
    return lower_limit, upper_limit

In [5]:
# we find the limits

lower_limit, upper_limit = find_limits(X_train, "worst smoothness", 3)
lower_limit, upper_limit

(0.06356074164705164, 0.20149734880520967)

In [6]:
# Cap outliers

X_train["worst smoothness"].clip(lower=lower_limit, upper=upper_limit, inplace=True)
X_test["worst smoothness"].clip(lower=lower_limit, upper=upper_limit, inplace=True)

X_train["worst smoothness"].min(), X_train["worst smoothness"].max()

C:\Users\NOU\AppData\Local\Temp\ipykernel_8140\2339751569.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train["worst smoothness"].clip(lower=lower_limit, upper=upper_limit, inplace=True)
C:\Users\NOU\AppData\Local\Temp\ipykernel_8140\2339751569.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

(0.07117, 0.20149734880520967)

In [7]:
# 使用Feature-engine
# let's separate the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

X_train.shape, X_test.shape

((398, 30), (171, 30))

In [8]:
capper = Winsorizer(
    variables=["worst smoothness", "worst texture"],
    capping_method="gaussian",
    tail="both",
    fold=3,
)

capper.fit(X_train)

Winsorizer(fold=3, tail='both', variables=['worst smoothness', 'worst texture'])

In [9]:
capper.left_tail_caps_

{'worst smoothness': 0.06364743973736293, 'worst texture': 7.115307053129349}

In [10]:
capper.right_tail_caps_

{'worst smoothness': 0.2014106507148984, 'worst texture': 43.95373817300131}

In [11]:
X_train = capper.transform(X_train)
X_test = capper.transform(X_test)

X_train[capper.variables_].max()

worst smoothness     0.201411
worst texture       43.953738
dtype: float64